In [ ]:
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.optimisation.algorithms import FISTA
from cil.optimisation.functions import ZeroFunction, LeastSquares

from cil.processors import Binner, TransmissionAbsorptionConverter
from cil.plugins.astra import ProjectionOperator
from cil.plugins.astra import FBP

from cil.plugins.ccpi_regularisation.functions import FGP_TV

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

from cil.io import TIFFStackReader

import numpy as np
import scipy
import matplotlib.pyplot as plt
import math
import os



In [ ]:

#parameters are from the original paper/author clarification
src_to_det = 967.3209839
src_to_object = 295
tilt = 30. * np.pi / 180.
centre_of_rotation = 0.254 * 6.

mag = src_to_det / src_to_object 
object_offset_x = centre_of_rotation / mag

source_pos_y = -src_to_object
detector_pos_y = src_to_det-src_to_object
angles_list = -np.linspace(0, 360, 2513, endpoint=False)
num_pixels_x = 1596
num_pixels_y = 1148
pixel_size_xy = 0.254


ag = AcquisitionGeometry.create_Cone3D( source_position=[0.0,source_pos_y,0.0], \
                                        detector_position=[0.0,detector_pos_y,0.0],\
                                        rotation_axis_position=[object_offset_x,0,0],\
                                        rotation_axis_direction=[0,-np.sin(tilt), np.cos(tilt)] ) \
                        .set_angles(angles=angles_list, angle_unit='degree')\
                        .set_panel( num_pixels=[num_pixels_x, num_pixels_y], \
                                    pixel_size=pixel_size_xy,\
                                    origin='top-left')\
                        .set_labels(['angle','vertical','horizontal'])
print(ag)
show_geometry(ag)

In [ ]:
path_common = '/home/tpc56154/Data/LegoLaminography'
path = 'Lego_Lamino30deg_XTH/'

reader = TIFFStackReader(file_name=os.path.join(path_common, path))
acq_data_raw = reader.read_as_AcquisitionData(ag)


islicer(acq_data_raw, direction='angle',origin='upper-left')


In [ ]:
# Read in the flat-field and dark-field radiographs and apply shading correction to the data
tiffs = [   os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Dark_80kV85uA.tif'),
            os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Flat_80kV85uA.tif') ]

reader = TIFFStackReader(file_name=tiffs)
SC = reader.read()

acq_data_SC = (acq_data_raw-SC[0]) / (SC[1]-SC[0])
islicer(acq_data_SC, direction='angle',origin='upper-left')

del acq_data_raw

In [ ]:
converter = TransmissionAbsorptionConverter()
acq_data_atten = converter(acq_data_SC)
islicer(acq_data_atten, direction='angle',origin='upper-left')

del acq_data_SC


In [ ]:
binning = 4
acq_data_filtered = acq_data_atten.copy()
acq_data_filtered.fill(scipy.ndimage.gaussian_filter(acq_data_atten.array, [0,binning//2,binning//2]))

roi = {'vertical': (100, -100, binning), 
       'horizontal': (100, -100, binning)}

binner = Binner(roi)
acq_data_binned = binner(acq_data_filtered)
islicer(acq_data_binned, direction='angle',origin='upper-left')

del acq_data_atten
del acq_data_filtered


In [ ]:
acq_data_binned.reorder('astra')
acq_data = acq_data_binned
ag = acq_data_binned.geometry


In [ ]:

ig = ag.get_ImageGeometry()
ig.voxel_num_y=250
ig.voxel_num_z=100
ig.center_x = 5
ig.center_z = 3
show_geometry(ag, ig)

fbp = FBP(ig, ag)
FDK_reco = fbp(acq_data)


show2D(FDK_reco,slice_list=[('vertical',40),('horizontal_y',139)], title="FBP reconstruction", fix_range=(-0.02,0.07))


##%% Now we hate set up the geometry! Time to compare reconstructions. We'll run Fista iwith LS, and Fista with TV and non-negativity. Both will have a warm start from the FDK reconstuction so will need fewer iterations


In [ ]:
Projector = ProjectionOperator(ig, ag)
LS = LeastSquares(A=Projector, b=acq_data)
LS.L = 82282.29687500001 #set for this geomnetry/binning
print("Lipschitz constant =", LS.L)


In [ ]:
fista_LS = FISTA(initial=FDK_reco, f=LS, g=ZeroFunction(), max_iteration=1000, update_objective_interval=10)


In [ ]:
fista_LS.run(100)
LS_reco = fista_LS.get_output()

show2D(LS_reco,slice_list=[('vertical',40),('horizontal_y', 139)], title="LS reconstruction", fix_range=(-0.02,0.07))

plt.figure()
plt.semilogy(fista_LS.objective)
plt.title('FISTA LS criterion')
plt.show()


In [ ]:
alpha = 0.5
TV = alpha*FGP_TV(device='gpu')


In [ ]:
fista_TV = FISTA(initial=FDK_reco, f=LS, g=TV, max_iteration=1000, update_objective_interval=10)
fista_TV.update_objective_interval = 10


In [ ]:
fista_TV.run(100)
TV_reco = fista_TV.get_output()
show2D(TV_reco,slice_list=[('vertical',40),('horizontal_y',139)], title="TV reconstruction", fix_range=(-0.02,0.07))

plt.figure()
plt.semilogy(fista_TV.objective)
plt.title('FISTA TV criterion')
plt.show()



In [ ]:

h1 = islicer(FDK_reco,direction='horizontal_y', minmax=(-0.02,0.07))
h2 = islicer(LS_reco,direction='horizontal_y', minmax=(-0.02,0.07))
h3 = islicer(TV_reco,direction='horizontal_y', minmax=(-0.02,0.07))

link_islicer(h1,h2,h3)
print("fin")